# Big Bench List Function Evaluation Data Creation
- https://huggingface.co/datasets/AlanYky/big-bench-list-function-turing

In [1]:
!pip install git+https://github.com/google/BIG-bench.git

  Cloning https://github.com/google/BIG-bench.git to /tmp/pip-req-build-vjocw3_y
  Running command git clone --filter=blob:none --quiet https://github.com/google/BIG-bench.git /tmp/pip-req-build-vjocw3_y
  Resolved https://github.com/google/BIG-bench.git to commit 092b196c1f8f14a54bbc62f24759d43bde46dd3b
  Preparing metadata (setup.py) ... done
Processing //tmp/pip-req-build-vjocw3_y/bleurt/bleurt-b610120347ef22b494b6d69b4316e303f5932516.zip (from bigbench==0.0.1)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.5/164.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.3/354.3 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 11.5 MB/s eta 0:00:00


In [2]:
!pip install datasets

In [3]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import os
import seqio
import tensorflow_datasets as tfds
from bigbench.bbseqio import task_api
from bigbench.bbseqio import tasks
import pandas as pd

In [4]:
import numpy as np
import random
import json

## The Catelogies of Different List Function Tasks

- Filtering and Classfication by GPT-4
- We just pick the tasks above the 0.5 human accuracy mean
- 157 tasks in total
- https://github.com/google/BIG-bench/tree/main/bigbench/benchmark_tasks/list_functions

In [ ]:
tasks = {
    "remove_elements": ["c170", "c121", "c061", "c048", "c120", "c127", "c126", "c137", "c238", "c095", "c049", "c062", "c081", "c002", "c132", "c225", "c030", "c007", "c075", "c053", "c064", "c135", "c025", "c122", "c003"],
    "modify_the_list": ["c072", "c151", "c080", "c189", "c147", "c100", "c145", "c079", "c038", "c022", "c212", "c021", "c105", "c070", "c052", "c190", "c222", "c107", "c011", "c104", "c192", "c182", "c046", "c106", "c114", "c093", "c195", "c196", "c224", "c116", "c140", "c071", "c161", "c068", "c067", "c112", "c248", "c199", "c176", "c066", "c053", "c119", "c198", "c219", "c177", "c125", "c211", "c087", "c013", "c069", "c117", "c228", "c197", "c065", "c156", "c110", "c233", "c230", "c020", "c235", "c157", "c185", "c213"],
    "input_independent": ["c102", "c045", "c043", "c042", "c041", "c044", "c090", "c091", "c097", "c092", "c077", "c006", "c001", "c103"],
    "add_elements": ["c050", "c038", "c022", "c212", "c021", "c046", "c114", "c093", "c196", "c067", "c065"],
    "miscellaneous": ["c223", "c108", "c187", "c105", "c070", "c052", "c044", "c190", "c037", "c222", "c107", "c011", "c104", "c192", "c182", "c106", "c195", "c116", "c068", "c112", "c248", "c199", "c176", "c066", "c119", "c198", "c219", "c177", "c125", "c211", "c087", "c013", "c069", "c117", "c228", "c197", "c156", "c110", "c233", "c230", "c235", "c157", "c185", "c213"]
}

In [5]:
tasks = {
    "remove_elements": ["c048", "c061", "c120", "c121", "c127", "c170"],
    "modify_the_list": ["c045", "c072", "c080", "c100", "c102", "c145", "c147", "c151"],
    "input_independent": ["c042", "c043"],
    "add_elements": ["c038", "c050"],
    "miscellaneous": ["c079", "c189"]
}

## Extract the List Functions Tasks
- We aims to take 3-shots example for evaluation
- We aims to find out the three important metrics for our evaluation including "question", "answer", and "task"

In [16]:
def extract_examples(categories, num_examples=5):
    # Initialize a list to hold all the data.
    data = []

    # Iterate over each category and its tasks.
    for category, tasks in categories.items():
        print(f"Processing {category}")
        for task_id in tasks:
            # Retrieve the dataset for this task.
            task = seqio.get_mixture_or_task(
              f'bigbench:list_functions.gen.t5_default_vocab.3_shot.all_examples.{task_id}')
            sequence_length = {"inputs": 32, "targets": 32}
            ds = task.get_dataset(
                sequence_length=sequence_length,  #  Trim long token sequences.
                split="all",  # Available splits = "all", "train", "validation".
            )

            for d in tfds.as_numpy(ds.take(num_examples)):
              # Extract the relevant fields.
              inputs = d['inputs_pretokenized'].decode('utf-8')
              targets = d['targets_pretokenized'].decode('utf-8')
              task_name = task_id

              # print(inputs)
              # print("\n")
              # print(targets)

              # Append to the data list.
              data.append([inputs, targets, task_name, category])

    # Create a DataFrame from the collected data.
    df = pd.DataFrame(data, columns=['inputs', 'targets', 'task_name', 'category'])

    return df

In [19]:
df = extract_examples(tasks, num_examples=15)

Processing remove_elements


Processing modify_the_list


Processing input_independent


Processing add_elements


Processing miscellaneous


In [20]:
df

,inputs,targets,task_name,category
0,Apply a function to the final input list to ge...,[1],c048,remove_elements
1,Apply a function to the final input list to ge...,[7],c048,remove_elements
2,Apply a function to the final input list to ge...,[4],c048,remove_elements
3,Apply a function to the final input list to ge...,[4],c048,remove_elements
4,Apply a function to the final input list to ge...,[0],c048,remove_elements
...,...,...,...,...
295,Apply a function to the final input list to ge...,"[0, 1, 2, 3]",c189,miscellaneous
296,Apply a function to the final input list to ge...,"[3, 4, 5, 6, 7, 8]",c189,miscellaneous
297,Apply a function to the final input list to ge...,"[91, 92]",c189,miscellaneous
298,Apply a function to the final input list to ge...,"[57, 58, 59, 60, 61, 62, 63, 64, 65]",c189,miscellaneous


from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('category').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

### Remove the Instruction

In [21]:
# Replace the specific instruction with an empty string in the 'inputs' column
df['inputs'] = df['inputs'].str.replace("Apply a function to the final input list to generate the output list. Use any preceding inputs and outputs as examples.\n", "", regex=False)

# If you want to ensure that any leading or trailing whitespace is also removed after the replacement,
# you can chain the `str.strip` method:
df['inputs'] = df['inputs'].str.strip()

df

,inputs,targets,task_name,category
0,"Q: [6, 7, 2, 3, 4, 0, 5, 9, 1, 8]\nA: [6]\n\nQ...",[1],c048,remove_elements
1,"Q: [6, 0, 9, 3, 8, 1, 4, 2, 5]\nA: [6]\n\nQ: [...",[7],c048,remove_elements
2,"Q: [7, 6, 5, 9, 0, 4, 2, 3]\nA: [7]\n\nQ: [1, ...",[4],c048,remove_elements
3,"Q: [4, 3, 8, 7, 9, 2, 5, 1, 0, 6]\nA: [4]\n\nQ...",[4],c048,remove_elements
4,"Q: [1, 8, 7, 9, 0, 3]\nA: [1]\n\nQ: [4, 3, 8, ...",[0],c048,remove_elements
...,...,...,...,...
295,"Q: [3, 1, 3, 1]\nA: [1, 2, 3]\n\nQ: [72, 76, 7...","[0, 1, 2, 3]",c189,miscellaneous
296,"Q: [2, 6, 3, 2]\nA: [2, 3, 4, 5, 6]\n\nQ: [16,...","[3, 4, 5, 6, 7, 8]",c189,miscellaneous
297,"Q: [3, 0, 3, 0, 3, 3, 3]\nA: [0, 1, 2, 3]\n\nQ...","[91, 92]",c189,miscellaneous
298,"Q: [2, 2, 2, 7, 7, 15, 7, 14]\nA: [2, 3, 4, 5,...","[57, 58, 59, 60, 61, 62, 63, 64, 65]",c189,miscellaneous


### Add Variation to Data

In [22]:
prefix_variations = [
    ("Student:", "Teacher:"),
    ("User:", "Assistant:"),
    ("User:", "Assistant:"),
    ("User:", "Assistant:"),
    ("Q:", "A:"),
    ("Input:", "Label:"),
    ("Question:", "Answer:")
]

def replace_prefixes(text):
    # Randomly select a new prefix pair
    new_prefixes = random.choice(prefix_variations)
    # Replace "Q:" with the new question prefix and "A:" with the new answer prefix
    new_text = text.replace("Q:", new_prefixes[0]).replace("A:", new_prefixes[1])
    return new_text

In [23]:
# Apply the function to each row in the 'inputs' column
df['inputs'] = df['inputs'].apply(replace_prefixes)

In [24]:
df

,inputs,targets,task_name,category
0,"Input: [6, 7, 2, 3, 4, 0, 5, 9, 1, 8]\nLabel: ...",[1],c048,remove_elements
1,"User: [6, 0, 9, 3, 8, 1, 4, 2, 5]\nAssistant: ...",[7],c048,remove_elements
2,"User: [7, 6, 5, 9, 0, 4, 2, 3]\nAssistant: [7]...",[4],c048,remove_elements
3,"User: [4, 3, 8, 7, 9, 2, 5, 1, 0, 6]\nAssistan...",[4],c048,remove_elements
4,"User: [1, 8, 7, 9, 0, 3]\nAssistant: [1]\n\nUs...",[0],c048,remove_elements
...,...,...,...,...
295,"User: [3, 1, 3, 1]\nAssistant: [1, 2, 3]\n\nUs...","[0, 1, 2, 3]",c189,miscellaneous
296,"User: [2, 6, 3, 2]\nAssistant: [2, 3, 4, 5, 6]...","[3, 4, 5, 6, 7, 8]",c189,miscellaneous
297,"User: [3, 0, 3, 0, 3, 3, 3]\nAssistant: [0, 1,...","[91, 92]",c189,miscellaneous
298,"User: [2, 2, 2, 7, 7, 15, 7, 14]\nAssistant: [...","[57, 58, 59, 60, 61, 62, 63, 64, 65]",c189,miscellaneous


### Add the Turing Data

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
turing_file = "/content/drive/MyDrive/Phi-2/data/turing/task.json"

# Reading the JSON file
with open(turing_file, 'r') as file:
    turing_data = json.load(file)

In [27]:
random.seed(42)
turing_examples = turing_data['examples']
random.shuffle(turing_examples)

first_50_turing_examples = turing_examples[:30]
first_50_turing_examples

[{'input': "Input: '0'\nOutput: ''\n\nInput: '00'\nOutput: '10'\n\nInput: '01'\nOutput: '10'\n\nInput: '0000'\nOutput: '10'\n\nInput: '11001'\nOutput:",
  'target': "''",
  'comment': "['>[<+o]', 6, 4]"},
 {'input': "Input: ''\nOutput: '000'\n\nInput: '0'\nOutput: ''\n\nInput: '00'\nOutput: ''\n\nInput: '110'\nOutput: ''\n\nInput: '11001'\nOutput:",
  'target': "''",
  'comment': "['-[-ooo]', 7, 4]"},
 {'input': "Input: ''\nOutput: '000'\n\nInput: '1'\nOutput: '0'\n\nInput: '100'\nOutput: '1'\n\nInput: '11'\nOutput: ''\n\nInput: '101'\nOutput:",
  'target': "'1'",
  'comment': "['>+o<+oo', 7, 4]"},
 {'input': "Input: '0'\nOutput: ''\n\nInput: '01'\nOutput: '00'\n\nInput: '0111'\nOutput: '00'\n\nInput: '011'\nOutput: '00'\n\nInput: '11101'\nOutput:",
  'target': "'11'",
  'comment': "['>[<oo]', 6, 4]"},
 {'input': "Input: ''\nOutput: '011'\n\nInput: '1'\nOutput: '1'\n\nInput: '00'\nOutput: '11'\n\nInput: '111'\nOutput: '1'\n\nInput: '11001'\nOutput:",
  'target': "'1'",
  'comment': "['

In [30]:
data_turing = []

for item in first_50_turing_examples:
  inputs = item["input"]
  targets = item["target"]
  task = "Turing"
  category = "Turing"
  data_turing.append([inputs, targets, task, category])

In [31]:
df_turing = pd.DataFrame(data_turing, columns=['inputs', 'targets', 'task_name', 'category'])
df_turing

,inputs,targets,task_name,category
0,Input: '0'\nOutput: ''\n\nInput: '00'\nOutput:...,'',Turing,Turing
1,Input: ''\nOutput: '000'\n\nInput: '0'\nOutput...,'',Turing,Turing
2,Input: ''\nOutput: '000'\n\nInput: '1'\nOutput...,'1',Turing,Turing
3,Input: '0'\nOutput: ''\n\nInput: '01'\nOutput:...,'11',Turing,Turing
4,Input: ''\nOutput: '011'\n\nInput: '1'\nOutput...,'1',Turing,Turing
5,Input: '00'\nOutput: ''\n\nInput: '000'\nOutpu...,'01',Turing,Turing
6,Input: ''\nOutput: '1'\n\nInput: '0'\nOutput: ...,'1',Turing,Turing
7,Input: '00'\nOutput: '0'\n\nInput: '10'\nOutpu...,'11',Turing,Turing
8,Input: ''\nOutput: '1'\n\nInput: '0'\nOutput: ...,'',Turing,Turing
9,Input: ''\nOutput: '1'\n\nInput: '00'\nOutput:...,'10',Turing,Turing


### Push to Hub

In [32]:
df_final = df.append(df_turing, ignore_index=True)

<ipython-input-32-7fabab0ef423>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df.append(df_turing, ignore_index=True)


In [33]:
df_final.shape

(330, 4)

In [34]:
df_final

,inputs,targets,task_name,category
0,"Input: [6, 7, 2, 3, 4, 0, 5, 9, 1, 8]\nLabel: ...",[1],c048,remove_elements
1,"User: [6, 0, 9, 3, 8, 1, 4, 2, 5]\nAssistant: ...",[7],c048,remove_elements
2,"User: [7, 6, 5, 9, 0, 4, 2, 3]\nAssistant: [7]...",[4],c048,remove_elements
3,"User: [4, 3, 8, 7, 9, 2, 5, 1, 0, 6]\nAssistan...",[4],c048,remove_elements
4,"User: [1, 8, 7, 9, 0, 3]\nAssistant: [1]\n\nUs...",[0],c048,remove_elements
...,...,...,...,...
325,Input: '0'\nOutput: '0'\n\nInput: '01'\nOutput...,'0',Turing,Turing
326,Input: '00'\nOutput: ''\n\nInput: '000'\nOutpu...,'',Turing,Turing
327,Input: ''\nOutput: '1'\n\nInput: '00'\nOutput:...,'0',Turing,Turing
328,Input: '10'\nOutput: '010'\n\nInput: '01'\nOut...,'111',Turing,Turing


In [35]:
from huggingface_hub import notebook_login

notebook_login()

In [36]:
from datasets import Dataset

eval_dataset = Dataset.from_pandas(df_final)

In [37]:
eval_dataset

Dataset({
    features: ['inputs', 'targets', 'task_name', 'category'],
    num_rows: 330
})

In [38]:
from huggingface_hub import create_repo

repo_name = "big-bench-list-function-turing-330"  # Choose a unique name
create_repo(repo_name, repo_type="dataset", private=False)  # Set `private=True` for a private dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


RepoUrl('https://huggingface.co/datasets/AlanYky/big-bench-list-function-turing-330', endpoint='https://huggingface.co', repo_type='dataset', repo_id='AlanYky/big-bench-list-function-turing-330')

In [39]:
# Push the train dataset
eval_dataset.push_to_hub("AlanYky/big-bench-list-function-turing-330")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AlanYky/big-bench-list-function-turing-330/commit/0a1bd1a7957d6d46b3c8d338382b27d54e1b8e2e', commit_message='Upload dataset', commit_description='', oid='0a1bd1a7957d6d46b3c8d338382b27d54e1b8e2e', pr_url=None, pr_revision=None, pr_num=None)